<a href="https://colab.research.google.com/github/jeremiahoclark/open_source_colabs/blob/main/fish_speech_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fish Speech - Colab Version

This notebook has been adapted for Google Colab from the original Fish Speech repository.

## Initial Setup

First, let's install the required dependencies and clone the repository.

In [1]:
# Install required packages
!pip install torch torchaudio transformers accelerate gradio

# Clone the Fish Speech repository
!git clone https://github.com/fishaudio/fish-speech.git
!cd fish-speech

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7
Clonin

## Set UTF-8 Encoding

In [2]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

## Download Model Checkpoints

Now we'll download the model checkpoints from Hugging Face.

In [3]:
# Install Hugging Face CLI if not already installed
!pip install -q huggingface_hub

# Download the model
!huggingface-cli download fishaudio/fish-speech-1.4 --local-dir checkpoints/fish-speech-1.4/

Fetching 8 files:   0% 0/8 [00:00<?, ?it/s]Downloading '.gitattributes' to 'checkpoints/fish-speech-1.4/.cache/huggingface/download/.gitattributes.a6344aac8c09253b3b630fb776ae94478aa0275b.incomplete'

.gitattributes: 100% 1.52k/1.52k [00:00<00:00, 7.49MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.4/.gitattributes
Fetching 8 files:  12% 1/8 [00:00<00:02,  2.61it/s]
README.md: 100% 1.39k/1.39k [00:00<00:00, 8.83MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.4/README.md

config.json: 100% 493/493 [00:00<00:00, 3.33MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.4/config.json

special_tokens_map.json: 100% 449/449 [00:00<00:00, 2.89MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.4/special_tokens_map.json

tokenizer_config.json: 100% 1.85k/1.85k [00:00<00:00, 15.0MB/s]
Download complete. Moving file to checkpoints/fish-speech-1.4/tokenizer_config.json

model.pth:   0% 0.00/989M [00:00<?, ?B/s]

(…)fly-gan-vq-fsq-8x10

## WebUI Inference

Run the WebUI with Colab's port forwarding capabilities.

In [4]:
# For Colab, we need to use port forwarding
from google.colab import output
output.serve_kernel_port_as_window(7860)

!python tools/webui.py \
    --llama-checkpoint-path checkpoints/fish-speech-1.4 \
    --decoder-checkpoint-path checkpoints/fish-speech-1.4/firefly-gan-vq-fsq-8x1024-21hz-generator.pth \
    --share  # This enables Gradio's public URL feature

Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

python3: can't open file '/content/tools/webui.py': [Errno 2] No such file or directory


## Break-down CLI Inference

### 1. Encode Reference Audio

In [6]:
# Upload your audio file using Colab's file uploader
from google.colab import files
uploaded = files.upload()
src_audio = next(iter(uploaded.keys()))  # Get the filename of the uploaded file

!python tools/vqgan/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/fish-speech-1.4/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"



Saving jap_speaker_1.wav to jap_speaker_1.wav
python3: can't open file '/content/tools/vqgan/inference.py': [Errno 2] No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'fake.wav'

In [7]:
from IPython.display import Audio, display
audio = Audio(filename="jap_speaker_1.wav")
display(audio)

### 2. Generate Semantic Tokens from Text

In [ ]:
!python tools/llama/generate.py \
    --text """
    この動画を 作成してくれてありがとうございました！
    とても聞きやすいです
    私の好きな日本の食べ物は油そばです。大好きです！
    次の動画を見るのを楽しみにしています。
    """ \
    --prompt-text "The text corresponding to reference audio" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.4" \
    --num-samples 2

### 3. Generate Speech from Semantic Tokens

In [ ]:
!python tools/vqgan/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.4/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="jap_speaker_1.wav")
display(audio)

python3: can't open file '/content/tools/vqgan/inference.py': [Errno 2] No such file or directory
